In [ ]:
# Import Libraries
import cv2
import numpy as np
import pandas as pd
import os

from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model,Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
import keras
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
# Following are the SOTA Pretrained Models available for transfer learning 
from tensorflow.keras.applications import VGG16,ResNet101,ResNet101V2, InceptionResNetV2,NASNetLarge,Xception

In [ ]:
cd /

In [ ]:
cd kaggle/input/hackerearth-deep-learning-identify-the-snake-breed/dataset

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
# Small Function to add file extension to id for easy reading from directory
def addext(nm):
    return nm+'.jpg'

In [ ]:
train['image_id'] = train['image_id'].apply(addext)

In [ ]:
test['image_id'] = test['image_id'].apply(addext)

In [ ]:
# Data augmentor
datagen = ImageDataGenerator(rescale=1/255,
        #validation_split=0.10,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest') 

In [ ]:
train_datagen = datagen.flow_from_dataframe(
        dataframe=train,
        directory="./train/",
        x_col="image_id",
        y_col="breed",
        subset="training",
        batch_size=54,
        seed=42,
        shuffle=True,
        color_mode="rgb", # Colour images are used.
        class_mode="categorical", # this does the work of encoding
        target_size=(224,224)) # Change acc to model requirments

In [ ]:
valid_datagen = datagen.flow_from_dataframe(
        dataframe=train,
        directory="./train/",
        x_col="image_id",
        y_col="breed",
        subset="validation",
        batch_size=54,
        seed=42,
        shuffle=True,
        color_mode="rgb",
        class_mode="categorical", # this does the work of encoding
        target_size=(224,224))

In [ ]:
# Series of transfer learning models, we do not implement the top layer, we add the layer ourselves acc to number of classes we have
# and train just the last layer 
#base_model = EfficientNetB7(weights='imagenet',input_shape=(224,224,3),include_top=False,pooling='avg')
#base_model = ResNet101(weights='imagenet',input_shape=(224,224,3),include_top=False,pooling='avg')
base_model = ResNet101V2(weights='imagenet',input_shape=(224,224,3),include_top=False,pooling='avg')
#base_model = Xception(weights='imagenet',input_shape=(224,224,3),include_top=False,pooling='avg')
#base_model = NASNetLarge(weights='imagenet',input_shape=(224,224,3),include_top=False,pooling='avg')
#base_model = InceptionResNetV2(weights='imagenet',input_shape=(224,224,3),include_top=False,pooling='avg')

# This is a try to train not just the last but last 3 layers of the network, this has to be done very carefully at a very low learning rate
for layer in base_model.layers[:-3]:
    layer.trainable = False
for layer in base_model.layers[-3:]:
    layer.trainable = True
model = Sequential([
  base_model, # Pretrained model
  # Last layers that we add manually
  Dense(1024, activation='relu'),
  # Dropout imporves generalisation
  Dropout(0.3),
  Dense(512,activation = 'relu'),
  Dropout(0.3),
  Dense(256, activation='relu'),
  Dropout(0.3),
  Dense(128, activation = 'relu'),
  Dense(35, activation='softmax'),
])

In [ ]:
# RMSprop worked the best for this model
optimizer = keras.optimizers.RMSprop(learning_rate=0.0008, decay = 1e-7)
model.compile(optimizer=optimizer,loss ="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
model.fit(train_datagen,epochs=20, 
                    #validation_data=valid_datagen, validation_steps=20,
                    steps_per_epoch=81)

In [ ]:
test_df=pd.read_csv('test.csv')
test_df["image_id"]=test_df["image_id"].apply(addext)
test_datagen = ImageDataGenerator(rescale=1./255,)
test_generator=test_datagen.flow_from_dataframe(dataframe=test_df,
                                            directory="test/",
                                            x_col="image_id",
                                            y_col=None,
                                            batch_size=128,
                                            seed=42,
                                            shuffle=False,
                                            class_mode=None,
                                            target_size=(331,331))


In [ ]:
y_pred=model.predict_generator(test_generator,verbose=1)
y_pred_2=np.argmax(y_pred,axis=1)
labels = (train_datagen.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in y_pred_2]
test_df=pd.read_csv('test.csv')
test_name=test_df['image_id'].values
data=pd.DataFrame((zip(test_name,predictions)),columns=['image_id','breed'])

In [ ]:
cd /

In [ ]:
cd kaggle/working

In [ ]:
data.to_csv('resnet101V2.csv',index = False)

In [ ]:
cd /